# Prediction with .h5 and .tflite file

### Imports


In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
import numpy as np
#import keras
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler


import tools

# Root CSV files directory
dirname = "./prediction_data/"
#dirname = "./data/"

# Frame count
frames = 100

In [2]:
model = tf.keras.models.load_model('sign_lang_recognition.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           38656     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 32)           12416     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 59,557
Trainable params: 59,557
Non-trainable params: 0
_________________________________________________________________


#### Load the file to predict and tokenize labels for comparison

In [4]:
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train=np.array(features, dtype="float32")

with open('tokens_json.txt', 'r') as outfile:
    json_ex = outfile.read()
    
#tokenizer = tools.tokenize(dirname)
tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(json_ex)
token_labels = {y:x for x,y in tokenizer.word_index.items()}
print(token_labels)

{1: 'computer', 2: 'deutschland', 3: 'hallo', 4: 'welt'}


In [ ]:
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))

In [ ]:
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
x_train=np.array(features, dtype="float32")
print(len(x_train[2]))

In [ ]:
print(data[2][0])

In [ ]:
y_pred = model.predict(x_train)

In [ ]:
predictions = np.array([np.argmax(pred) for pred in y_pred])
print(predictions)
print([token_labels[p] for p in predictions])


In [ ]:
len(predictions)

https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python

In [ ]:
import numpy as np
import tensorflow as tf
from time import time

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="sign_lang_recognition.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
# Test model on random input data.

#interpreter.set_tensor(0.0, x_train)

input_shape = input_details[0]['shape']
time1= time()
input_data = [1]
predictions = []
for x_pred in x_train:
    input_data[0] = x_pred

    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(np.argmax(output_data))
predictions = [token_labels[p] for p in predictions]
print("Predictions:", predictions)
time2= time()
dauer = time2 - time1
print("Anzahl Samples: ", len(predictions))
print("Predictiondauer: ", dauer)
print("Samples pro Sekunde:", (len(predictions)/dauer))